In [1]:
import cv2
import os

# Save dataset path
dataset_path = r"C:\Users\sagni\Downloads\Face Concentration\dataset"

# Create dataset folder if it doesn't exist
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Initialize face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start video capture
cap = cv2.VideoCapture(0)

# Get user name for labeling
name = input("Enter the name of the person: ").strip()
person_dir = os.path.join(dataset_path, name)
if not os.path.exists(person_dir):
    os.makedirs(person_dir)

print(f"📸 Capturing images for {name}... (Press 'q' to quit early)")

count = 0
MAX_IMAGES = 50  # Number of images per person

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y + h, x:x + w]
        img_path = os.path.join(person_dir, f"{count + 1}.jpg")
        cv2.imwrite(img_path, face_img)
        count += 1
        print(f"✅ Saved image {count}/{MAX_IMAGES}")

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{name}: {count}/{MAX_IMAGES}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Collecting Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= MAX_IMAGES:
        break

cap.release()
cv2.destroyAllWindows()
print(f"🎯 Dataset collection completed for {name}!")


Enter the name of the person:  sagnik


📸 Capturing images for sagnik... (Press 'q' to quit early)
✅ Saved image 1/50
✅ Saved image 2/50
✅ Saved image 3/50
✅ Saved image 4/50
✅ Saved image 5/50
✅ Saved image 6/50
✅ Saved image 7/50
✅ Saved image 8/50
✅ Saved image 9/50
✅ Saved image 10/50
✅ Saved image 11/50
✅ Saved image 12/50
✅ Saved image 13/50
✅ Saved image 14/50
✅ Saved image 15/50
✅ Saved image 16/50
✅ Saved image 17/50
✅ Saved image 18/50
✅ Saved image 19/50
✅ Saved image 20/50
✅ Saved image 21/50
✅ Saved image 22/50
✅ Saved image 23/50
✅ Saved image 24/50
✅ Saved image 25/50
✅ Saved image 26/50
✅ Saved image 27/50
✅ Saved image 28/50
✅ Saved image 29/50
✅ Saved image 30/50
✅ Saved image 31/50
✅ Saved image 32/50
✅ Saved image 33/50
✅ Saved image 34/50
✅ Saved image 35/50
✅ Saved image 36/50
✅ Saved image 37/50
✅ Saved image 38/50
✅ Saved image 39/50
✅ Saved image 40/50
✅ Saved image 41/50
✅ Saved image 42/50
✅ Saved image 43/50
✅ Saved image 44/50
✅ Saved image 45/50
✅ Saved image 46/50
✅ Saved image 47/50
✅ Saved im

In [2]:
import cv2
import os
import numpy as np
from PIL import Image
import pickle

# Path to dataset
BASE_DIR = r"C:\Users\sagni\Downloads\Face Concentration\Face Recognization"
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
x_train = []
y_labels = []

for root, dirs, files in os.walk(BASE_DIR):
    for file in files:
        if file.endswith(("png", "jpg", "jpeg")):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "_").lower()

            if label not in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]

            pil_image = Image.open(path).convert("L")  # Grayscale
            image_array = np.array(pil_image, "uint8")

            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=5)

            for (x, y, w, h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

# Save labels
with open(os.path.join(BASE_DIR, "labels.pickle"), 'wb') as f:
    pickle.dump(label_ids, f)

# Train and save model
recognizer.train(x_train, np.array(y_labels))
recognizer.save(os.path.join(BASE_DIR, "trainer.yml"))

print("✅ Training complete. Model and labels saved.")


✅ Training complete. Model and labels saved.


In [ ]:
import cv2
import pickle
import os
from datetime import datetime

# Paths
MODEL_PATH = r"C:\Users\sagni\Downloads\Face Concentration\models"
DATASET_PATH = r"C:\Users\sagni\Downloads\Face Concentration\Face Recognization"
ALERT_FOLDER = os.path.join(DATASET_PATH, "violations")

if not os.path.exists(ALERT_FOLDER):
    os.makedirs(ALERT_FOLDER)

# Load DNN Face Detector
dnn_net = cv2.dnn.readNetFromCaffe(
    os.path.join(MODEL_PATH, "deploy.prototxt.txt"),
    os.path.join(MODEL_PATH, "res10_300x300_ssd_iter_140000.caffemodel")
)

# Load LBPH Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(os.path.join(DATASET_PATH, "trainer.yml"))

# Load Labels
with open(os.path.join(DATASET_PATH, "labels.pickle"), 'rb') as f:
    labels = pickle.load(f)
    labels = {v: k for k, v in labels.items()}  # Reverse key-value

# Save violation image
def save_violation(frame, name="unknown"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(ALERT_FOLDER, f"{name}_{timestamp}.jpg")
    cv2.imwrite(filename, frame)
    print(f"📸 Saved violation: {filename}")

# Show all violations after session
def show_all_violations():
    images = [f for f in os.listdir(ALERT_FOLDER) if f.endswith(".jpg")]
    if not images:
        print("✅ No violations recorded.")
        return

    print(f"🖼 Showing {len(images)} violation(s):")
    for img_file in images:
        img = cv2.imread(os.path.join(ALERT_FOLDER, img_file))
        cv2.imshow(img_file, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot access webcam")
    exit()

print("📷 Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare frame for DNN
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0)
    )
    dnn_net.setInput(blob)
    detections = dnn_net.forward()

    # Loop over detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter weak detections
        if confidence < 0.5:
            continue

        box = detections[0, 0, i, 3:7] * [w, h, w, h]
        (startX, startY, endX, endY) = box.astype("int")

        face_roi_gray = cv2.cvtColor(frame[startY:endY, startX:endX], cv2.COLOR_BGR2GRAY)
        face_roi_resized = cv2.resize(face_roi_gray, (200, 200))

        # Recognize face
        id_, conf = recognizer.predict(face_roi_resized)
        name = labels.get(id_, "Unknown")

        # Draw box & label
        color = (0, 255, 0) if conf >= 50 else (0, 0, 255)  # Green for known, Red for unknown/low confidence
        label_text = f"{name} ({round(conf, 2)}%)" if conf >= 50 else "Unknown"
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.putText(frame, label_text, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Save if unknown or low confidence (face turned, hidden, etc.)
        if conf < 50:
            save_violation(frame, name)

    # Show video feed
    cv2.imshow("Face Recognition + DNN", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
show_all_violations()


📷 Press 'q' to quit
📸 Saved violation: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\violations_20250708_133636.jpg
📸 Saved violation: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\violations_20250708_133636.jpg


In [ ]:
import cv2
import pickle
import os
from datetime import datetime
import uuid
import time

# Paths
MODEL_PATH = r"C:\Users\sagni\Downloads\Face Concentration\models"
DATASET_PATH = r"C:\Users\sagni\Downloads\Face Concentration\Face Recognization"
ALERT_FOLDER = os.path.join(DATASET_PATH, "violations")

if not os.path.exists(ALERT_FOLDER):
    os.makedirs(ALERT_FOLDER)

# Load DNN Face Detector
dnn_net = cv2.dnn.readNetFromCaffe(
    os.path.join(MODEL_PATH, "deploy.prototxt.txt"),
    os.path.join(MODEL_PATH, "res10_300x300_ssd_iter_140000.caffemodel")
)

# Load LBPH Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(os.path.join(DATASET_PATH, "trainer.yml"))

# Load Labels
with open(os.path.join(DATASET_PATH, "labels.pickle"), 'rb') as f:
    labels = pickle.load(f)
    labels = {v: k for k, v in labels.items()}  # Reverse key-value

# Track last saved times per name to avoid saving same face repeatedly
last_saved_time = {}
cooldown_seconds = 5  # Save violation for same person only once every 5 seconds

# Save violation image in separate folder per person
def save_violation(frame, name="Unknown"):
    now = time.time()
    if name in last_saved_time and (now - last_saved_time[name] < cooldown_seconds):
        return  # Skip saving (still in cooldown for this person)
    last_saved_time[name] = now

    # Create person-specific folder
    person_folder = os.path.join(ALERT_FOLDER, name)
    if not os.path.exists(person_folder):
        os.makedirs(person_folder)

    # Create unique filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # up to milliseconds
    unique_id = uuid.uuid4().hex[:6]  # Random 6-char ID
    filename = os.path.join(person_folder, f"{timestamp}_{unique_id}.jpg")
    cv2.imwrite(filename, frame)
    print(f"📸 Saved violation for {name}: {filename}")

# Show all violations after session
def show_all_violations():
    print("🖼 Showing all saved violations...")
    for person in os.listdir(ALERT_FOLDER):
        person_folder = os.path.join(ALERT_FOLDER, person)
        if not os.path.isdir(person_folder):
            continue
        print(f"👤 Violations for: {person}")
        images = [f for f in os.listdir(person_folder) if f.endswith(".jpg")]
        for img_file in images:
            img_path = os.path.join(person_folder, img_file)
            img = cv2.imread(img_path)
            cv2.imshow(f"{person}: {img_file}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot access webcam")
    exit()

print("📷 Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare frame for DNN
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0)
    )
    dnn_net.setInput(blob)
    detections = dnn_net.forward()

    # Loop over detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter weak detections
        if confidence < 0.5:
            continue

        box = detections[0, 0, i, 3:7] * [w, h, w, h]
        (startX, startY, endX, endY) = box.astype("int")

        # Get face region
        face_roi_gray = cv2.cvtColor(frame[startY:endY, startX:endX], cv2.COLOR_BGR2GRAY)
        face_roi_resized = cv2.resize(face_roi_gray, (200, 200))

        # Recognize face
        try:
            id_, conf = recognizer.predict(face_roi_resized)
            name = labels.get(id_, "Unknown")
        except:
            name = "Unknown"
            conf = 0

        # Draw box & label
        color = (0, 255, 0) if conf >= 50 else (0, 0, 255)  # Green = known, Red = unknown/low confidence
        label_text = f"{name} ({round(conf, 2)}%)" if conf >= 50 else "Unknown"
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.putText(frame, label_text, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Save if unknown or low confidence
        if conf < 50:
            save_violation(frame, name)

    # Show video feed
    cv2.imshow("Face Recognition + DNN", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
show_all_violations()


📷 Press 'q' to quit
📸 Saved violation for violations: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\violations\20250708_142936_664_b4b920.jpg
📸 Saved violation for alerts: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\alerts\20250708_142936_836_cb9775.jpg
📸 Saved violation for alerts: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\alerts\20250708_142941_931_891485.jpg
📸 Saved violation for violations: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\violations\20250708_142942_271_34e886.jpg
📸 Saved violation for alerts: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\alerts\20250708_142947_149_a0e29d.jpg
📸 Saved violation for alerts: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\alerts\20250708_142952_175_0d60ac.jpg
📸 Saved violation for alerts: C:\Users\sagni\Downloads\Face Concentration\Face Recognization\violations\alerts\2

In [2]:
!pip install mediapipe


  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   - -------------------------------------- 1.3/51.0 MB 8.4 MB/s eta 0:00:06
   - -------------------------------------- 2.1/51.0 MB 6.9 MB/s eta 0:00:08
   -- ------------------------------------- 3.1/51.0 MB 5.6 MB/s eta 0:00:09
   --- ------------------------------------ 4.2/51.0 MB 5.7 MB/s eta 0:00:09
   --- ------------------------------------ 5.0/51.0 MB 5.2 MB/s eta 0:00:09
   ---- ----------------------------------- 6.0/51.0 MB 5.0 MB/s eta 0:00:10
   ----- ---------------------------------- 6.8/51.0 MB 5.0 MB/s eta 0:00:09
   ------ --------------------------------- 7.9/51.0 MB 4.9 MB/s eta 0:00:09
   ------ --------------------------------- 8.9/51.0 MB 4.9 MB/s eta 0:00:09
   ------- -------------------------------- 10.0/51.0 MB 5.0 MB/s eta 0:00:09
   -------- ------------------------------- 11.3/51.0 MB 5.0 MB/s eta 0:00:08
   ----

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from ultralytics import YOLO
from datetime import datetime
import os
from collections import deque
import threading

# Load YOLO models
yolo_custom = YOLO(r"C:\Users\sagni\runs\detect\custom_phone_model4\weights\best.pt")
yolo_people = YOLO('yolov8n.pt')

print("📦 Custom model classes:", yolo_custom.names)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Eye and iris landmarks
LEFT_IRIS = 468
RIGHT_IRIS = 473

# Folder setup
alert_folder = "alerts"
eye_crop_folder = os.path.join(alert_folder, "eye_crops")
phone_crop_folder = os.path.join(alert_folder, "phone_crops")
os.makedirs(alert_folder, exist_ok=True)
os.makedirs(eye_crop_folder, exist_ok=True)
os.makedirs(phone_crop_folder, exist_ok=True)

# Global flags and counters
stop_flag = False
background_people_saved = set()
phone_violation_count = 0
reflection_violation_count = 0
background_people_count = 0
MAX_VIOLATIONS = 3  # Stop session after 3 total violations

def save_violation(frame, reason="violation"):
    """Save a violation screenshot with timestamp"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{alert_folder}/{reason}_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"📸 Saved {reason} screenshot: {filename}")
    return filename

def save_crop(crop, folder, prefix="crop"):
    """Save a cropped image for later training/debugging"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{folder}/{prefix}_{timestamp}.jpg"
    cv2.imwrite(filename, crop)
    print(f"📦 Saved {prefix} crop: {filename}")

def detect_custom_objects(frame):
    """Detect phones with custom YOLO model"""
    return yolo_custom.predict(frame, imgsz=960, conf=0.2, verbose=False)[0]

def detect_people(frame):
    """Detect people with YOLOv8n"""
    result = yolo_people.predict(frame, imgsz=960, conf=0.3, verbose=False)[0]
    people_boxes = []
    for box in result.boxes:
        cls_name = result.names[int(box.cls[0])].lower()
        if cls_name == "person":
            people_boxes.append(box.xyxy[0].cpu().numpy())  # xyxy: [x1, y1, x2, y2]
    return people_boxes

def get_primary_user_box(landmarks, w, h):
    """Get bounding box around the primary user's face"""
    xs = [lm.x * w for lm in landmarks]
    ys = [lm.y * h for lm in landmarks]
    x1, y1, x2, y2 = int(min(xs)), int(min(ys)), int(max(xs)), int(max(ys))
    padding = 80
    x1, y1 = max(0, x1 - padding), max(0, y1 - padding)
    x2, y2 = min(w, x2 + padding), min(h, y2 + padding)
    return np.array([x1, y1, x2, y2])

def is_outside_primary(box, primary_box):
    """Check if a detected person is outside the primary user box"""
    px1, py1, px2, py2 = primary_box
    bx1, by1, bx2, by2 = box
    overlap_x = max(0, min(px2, bx2) - max(px1, bx1))
    overlap_y = max(0, min(py2, by2) - max(py1, by1))
    overlap_area = overlap_x * overlap_y
    box_area = (bx2 - bx1) * (by2 - by1)
    return overlap_area / box_area < 0.5

def detect_phone_reflection(frame, landmarks, w, h):
    """Detect phones in the iris regions (reflections)"""
    detected = False
    for idx in [LEFT_IRIS, RIGHT_IRIS]:
        cx, cy = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
        r = 60  # Bigger crop radius for better detection
        eye_crop = frame[max(0, cy - r):cy + r, max(0, cx - r):cx + r]
        if eye_crop.shape[0] == 0 or eye_crop.shape[1] == 0:
            continue

        # Save crop for debug/training
        save_crop(eye_crop, eye_crop_folder, prefix="eye")

        # Resize for YOLO
        eye_crop_resized = cv2.resize(eye_crop, (128, 128))
        eye_res = yolo_custom.predict(eye_crop_resized, imgsz=128, conf=0.05, verbose=False)[0]
        for box in eye_res.boxes:
            cls_name = yolo_custom.names[int(box.cls[0])].lower()
            if cls_name in ['phone', 'mobile phone', 'cell phone', 'smartphone']:
                print("👁️ Reflection phone detected by YOLO:", cls_name)
                detected = True
                save_crop(eye_crop, phone_crop_folder, prefix="reflection")
                break
        if detected:
            break

        # Fallback: OpenCV template matching (optional)
        # (You can add template matching logic here if needed)

    return detected

def draw_alert(frame, text, color=(0, 0, 255)):
    """Draw alert text on frame"""
    cv2.putText(frame, text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)

def draw_counters(frame):
    """Draw live counters on frame"""
    cv2.putText(frame, f"Phones: {phone_violation_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Reflections: {reflection_violation_count}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f"Background: {background_people_count}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

def stop_camera_listener():
    """Stop camera when Enter is pressed"""
    global stop_flag
    input("🔘 Press Enter to stop camera...\n")
    stop_flag = True

def main():
    global stop_flag, phone_violation_count, reflection_violation_count, background_people_saved, background_people_count
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Webcam access failed")
        return

    threading.Thread(target=stop_camera_listener, daemon=True).start()

    try:
        while not stop_flag:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            h, w, _ = frame.shape
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_results = face_mesh.process(frame_rgb)

            # Detect people
            people_boxes = detect_people(frame)

            # Detect phones
            custom_results = detect_custom_objects(frame)
            custom_labels = [yolo_custom.names[int(box.cls[0])].lower() for box in custom_results.boxes]
            phone_detected = any(lbl in ['phone', 'mobile phone', 'cell phone'] for lbl in custom_labels)

            if phone_detected:
                phone_violation_count += 1
                draw_alert(frame, f"📱 PHONE DETECTED ({phone_violation_count})")
                save_violation(frame, "phone_detected")

            if face_results.multi_face_landmarks:
                landmarks = face_results.multi_face_landmarks[0].landmark
                primary_box = get_primary_user_box(landmarks, w, h)
                cv2.rectangle(frame, tuple(primary_box[:2]), tuple(primary_box[2:]), (0, 255, 0), 2)

                # Check background people
                for box in people_boxes:
                    bx1, by1, bx2, by2 = map(int, box)
                    if is_outside_primary(box, primary_box):
                        hash_id = f"{bx1}_{by1}_{bx2}_{by2}"
                        if hash_id not in background_people_saved:
                            background_people_saved.add(hash_id)
                            background_people_count += 1
                            draw_alert(frame, "⚠️ BACKGROUND PERSON DETECTED")
                            save_violation(frame, "background_person")

                # Detect phone reflection
                if detect_phone_reflection(frame, landmarks, w, h):
                    reflection_violation_count += 1
                    draw_alert(frame, f"👁️ REFLECTION DETECTED ({reflection_violation_count})", (0, 255, 255))
                    save_violation(frame, "reflection_detected")

            # Draw counters
            draw_counters(frame)

            # Stop session if too many violations
            total_violations = phone_violation_count + reflection_violation_count
            if total_violations >= MAX_VIOLATIONS:
                draw_alert(frame, "🛑 EXAM TERMINATED")
                save_violation(frame, "exam_terminated")
                cv2.imshow("Violation", frame)
                cv2.waitKey(3000)
                break

            # Show live frame
            cv2.imshow("Exam Monitor", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Session Ended")

main()


📦 Custom model classes: {0: 'phone'}
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152119.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152119.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152120.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152120.jpg
📸 Saved background_person screenshot: alerts/background_person_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152217.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152218.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152218.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152218.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152218.jpg
📦 Saved eye crop: alerts\eye_crops/eye_20250708_152218

🔘 Press Enter to stop camera...
 stop


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
from ultralytics import YOLO
from datetime import datetime
import os
import threading

# Load YOLO models
yolo_custom = YOLO(r"C:\Users\sagni\runs\detect\custom_phone_model4\weights\best.pt")
yolo_people = YOLO('yolov8s.pt')  # use YOLOv8 small model for people

print("📦 Custom model classes:", yolo_custom.names)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Eye and iris landmarks
LEFT_IRIS = 468
RIGHT_IRIS = 473

# Folder setup
alert_folder = "alerts"
os.makedirs(alert_folder, exist_ok=True)

# Global flags and counters
stop_flag = False
background_people_count = 0
phone_violation_count = 0
reflection_violation_count = 0
MAX_VIOLATIONS = 3

def save_violation(frame, reason="violation"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{alert_folder}/{reason}_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"📸 Saved {reason} screenshot: {filename}")

def detect_people(frame):
    """Detect people with YOLOv8s"""
    result = yolo_people.predict(frame, imgsz=1280, conf=0.3, verbose=False)[0]
    people_boxes = []
    for box in result.boxes:
        cls_name = result.names[int(box.cls[0])].lower()
        conf_score = box.conf[0].cpu().numpy()
        if cls_name == "person":
            people_boxes.append((box.xyxy[0].cpu().numpy(), conf_score))
    return people_boxes

def detect_custom_objects(frame, conf_threshold=0.2):
    """Detect phones with custom YOLO model"""
    result = yolo_custom.predict(frame, imgsz=1280, conf=conf_threshold, verbose=False)[0]
    phone_boxes = []
    for box in result.boxes:
        cls_name = yolo_custom.names[int(box.cls[0])].lower()
        conf_score = box.conf[0].cpu().numpy()
        phone_boxes.append((box.xyxy[0].cpu().numpy(), cls_name, conf_score))
    return phone_boxes

def detect_phone_reflection(frame, landmarks, w, h):
    """Detect phones in eye reflections"""
    for idx in [LEFT_IRIS, RIGHT_IRIS]:
        cx, cy = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
        r = 60  # Crop radius
        eye_crop = frame[max(0, cy - r):cy + r, max(0, cx - r):cx + r]
        if eye_crop.shape[0] == 0 or eye_crop.shape[1] == 0:
            continue
        eye_res = detect_custom_objects(eye_crop, conf_threshold=0.01)
        if eye_res:
            print("👁️ Phone reflection detected!")
            save_violation(eye_crop, "reflection_phone")
            return True
    return False

def stop_camera_listener():
    global stop_flag
    input("🔘 Press Enter to stop camera...\n")
    stop_flag = True

def main():
    global stop_flag, phone_violation_count, reflection_violation_count, background_people_count
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    if not cap.isOpened():
        print("❌ Webcam access failed")
        return

    threading.Thread(target=stop_camera_listener, daemon=True).start()

    while not stop_flag:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_results = face_mesh.process(frame_rgb)

        # Detect people
        people_boxes = detect_people(frame)
        for (box, conf) in people_boxes:
            (x1, y1, x2, y2) = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"Person {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Detect phones
        phone_boxes = detect_custom_objects(frame)
        for (box, cls_name, conf) in phone_boxes:
            (x1, y1, x2, y2) = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"{cls_name} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            phone_violation_count += 1
            save_violation(frame, "phone_detected")

        # Detect reflections
        if face_results.multi_face_landmarks:
            landmarks = face_results.multi_face_landmarks[0].landmark
            if detect_phone_reflection(frame, landmarks, w, h):
                reflection_violation_count += 1

        # Draw counters
        cv2.putText(frame, f"Phones: {phone_violation_count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"Reflections: {reflection_violation_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Stop session if too many violations
        total_violations = phone_violation_count + reflection_violation_count
        if total_violations >= MAX_VIOLATIONS:
            cv2.putText(frame, "🛑 SESSION TERMINATED", (50, h // 2),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            save_violation(frame, "session_terminated")
            cv2.imshow("Exam Monitor", frame)
            cv2.waitKey(5000)
            break

        cv2.imshow("Exam Monitor", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("✅ Session Ended")

main()


📦 Custom model classes: {0: 'phone'}
👁️ Phone reflection detected!
📸 Saved reflection_phone screenshot: alerts/reflection_phone_20250708_153054.jpg


🔘 Press Enter to stop camera...
 Stop


✅ Session Ended


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from ultralytics import YOLO
from datetime import datetime
import os
import threading

# Load YOLO models
yolo_custom = YOLO(r"C:\Users\sagni\runs\detect\custom_phone_model4\weights\best.pt")
yolo_general_phone = YOLO('yolov8s.pt')  # general YOLO for phones
yolo_people = YOLO('yolov8s.pt')         # YOLOv8 small model for people

print("📦 Custom model classes:", yolo_custom.names)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# Eye and iris landmarks
LEFT_IRIS = 468
RIGHT_IRIS = 473

# Folder setup
alert_folder = "alerts"
os.makedirs(alert_folder, exist_ok=True)

# Global flags and counters
stop_flag = False
object_tracker = {}  # To track unique objects
violation_log = []   # To store violation file paths
MAX_VIOLATIONS = 3
total_violation_count = 0

def save_violation(frame, reason="violation"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{alert_folder}/{reason}_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"📸 Saved {reason} screenshot: {filename}")
    violation_log.append(filename)
    return filename

def detect_people(frame):
    """Detect people with YOLOv8s"""
    result = yolo_people.predict(frame, imgsz=1280, conf=0.3, verbose=False)[0]
    people_boxes = []
    for box in result.boxes:
        cls_name = result.names[int(box.cls[0])].lower()
        if cls_name == "person":
            people_boxes.append(box.xyxy[0].cpu().numpy())
    return people_boxes

def detect_custom_objects(frame, conf_threshold=0.2):
    """Combine custom and general YOLO for phones"""
    phones = []
    # Custom YOLO
    custom_res = yolo_custom.predict(frame, imgsz=1280, conf=conf_threshold, verbose=False)[0]
    for box in custom_res.boxes:
        cls_name = yolo_custom.names[int(box.cls[0])].lower()
        phones.append((box.xyxy[0].cpu().numpy(), cls_name, box.conf[0].cpu().numpy()))
    # General YOLO
    general_res = yolo_general_phone.predict(frame, imgsz=1280, conf=0.3, verbose=False)[0]
    for box in general_res.boxes:
        cls_name = yolo_general_phone.names[int(box.cls[0])].lower()
        if cls_name in ['cell phone', 'mobile phone', 'phone', 'smartphone']:
            phones.append((box.xyxy[0].cpu().numpy(), cls_name, box.conf[0].cpu().numpy()))
    return phones

def detect_phone_reflection(frame, landmarks, w, h):
    """Detect phones in eye reflections"""
    for idx in [LEFT_IRIS, RIGHT_IRIS]:
        cx, cy = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
        r = 100  # Larger crop radius
        eye_crop = frame[max(0, cy - r):cy + r, max(0, cx - r):cx + r]
        if eye_crop.shape[0] < 50 or eye_crop.shape[1] < 50:
            continue
        eye_res = detect_custom_objects(eye_crop, conf_threshold=0.05)
        for (box, cls_name, conf) in eye_res:
            if cls_name in ['phone', 'mobile phone', 'cell phone', 'smartphone']:
                print(f"👁️ Reflection phone detected: {cls_name} ({conf:.2f})")
                save_violation(eye_crop, "reflection_phone")
                return True
    return False

def get_centroid(box):
    """Calculate centroid of a bounding box"""
    x1, y1, x2, y2 = box
    return ((x1 + x2) / 2, (y1 + y2) / 2)

def update_tracker(object_id, centroid):
    """Track unique objects using their centroid"""
    for oid, prev_centroid in object_tracker.items():
        dist = np.linalg.norm(np.array(centroid) - np.array(prev_centroid))
        if dist < 50:  # threshold for considering same object
            object_tracker[oid] = centroid
            return False  # already tracked
    object_tracker[object_id] = centroid
    return True  # new object

def stop_camera_listener():
    global stop_flag
    input("🔘 Press Enter to stop camera...\n")
    stop_flag = True

def show_violation_summary():
    """Display all violation thumbnails"""
    print(f"\n📸 Total Violations: {len(violation_log)}")
    for idx, img_path in enumerate(violation_log):
        img = cv2.imread(img_path)
        cv2.imshow(f"Violation {idx+1}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def main():
    global stop_flag, total_violation_count
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

    if not cap.isOpened():
        print("❌ Webcam access failed")
        return

    threading.Thread(target=stop_camera_listener, daemon=True).start()

    while not stop_flag:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_results = face_mesh.process(frame_rgb)

        # Detect people
        people_boxes = detect_people(frame)
        for box in people_boxes:
            (x1, y1, x2, y2) = map(int, box)
            centroid = get_centroid((x1, y1, x2, y2))
            if update_tracker(f"person_{centroid}", centroid):
                save_violation(frame, "background_person")
                total_violation_count += 1
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Detect phones
        phone_boxes = detect_custom_objects(frame)
        for (box, cls_name, conf) in phone_boxes:
            (x1, y1, x2, y2) = map(int, box)
            centroid = get_centroid((x1, y1, x2, y2))
            if update_tracker(f"phone_{centroid}", centroid):
                save_violation(frame, "phone_detected")
                total_violation_count += 1
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"{cls_name} ({conf:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Detect reflections
        if face_results.multi_face_landmarks:
            landmarks = face_results.multi_face_landmarks[0].landmark
            if detect_phone_reflection(frame, landmarks, w, h):
                total_violation_count += 1

        # Stop session if too many violations
        if total_violation_count >= MAX_VIOLATIONS:
            cv2.putText(frame, "🛑 SESSION TERMINATED", (50, h // 2),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            save_violation(frame, "session_terminated")
            cv2.imshow("Exam Monitor", frame)
            cv2.waitKey(3000)
            break

        cv2.imshow("Exam Monitor", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    show_violation_summary()
    print("✅ Session Ended")

main()


📦 Custom model classes: {0: 'phone'}
📸 Saved background_person screenshot: alerts/background_person_20250708_154346.jpg
📸 Saved background_person screenshot: alerts/background_person_20250708_154347.jpg
📸 Saved background_person screenshot: alerts/background_person_20250708_154357.jpg
📸 Saved session_terminated screenshot: alerts/session_terminated_20250708_154359.jpg

📸 Total Violations: 4
